In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import datetime
from datetime import datetime
import seaborn as sns
from datetime import datetime, timedelta
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM
import math
from sklearn.metrics import mean_squared_error

**Load FAANG Stock Datasets**

In [2]:
from google.colab import files
data_to_load1 = files.upload()
data_to_load2 = files.upload()
data_to_load3 = files.upload()
data_to_load4 = files.upload()
data_to_load5 = files.upload()

Saving Amazon_Historical.csv to Amazon_Historical.csv


Saving Apple_Historical.csv to Apple_Historical.csv


Saving Meta_Historical.csv to Meta_Historical.csv


Saving Google_Alphabet_Historical.csv to Google_Alphabet_Historical.csv


Saving Netflix_Historical.csv to Netflix_Historical.csv


**Read FAANG Stock Datasets in DataFrame**

In [3]:
import io
df_Amazon = pd.read_csv(io.BytesIO(data_to_load1['Amazon_Historical.csv']))
df_Apple = pd.read_csv(io.BytesIO(data_to_load2['Apple_Historical.csv']))
df_Meta = pd.read_csv(io.BytesIO(data_to_load3['Meta_Historical.csv']))
df_Google = pd.read_csv(io.BytesIO(data_to_load4['Google_Alphabet_Historical.csv']))
df_Netflix = pd.read_csv(io.BytesIO(data_to_load5['Netflix_Historical.csv']))

**AMAZON Prediction**

In [11]:
df_Amazon['Close/Last'] = df_Amazon['Close/Last'].replace('[\$,]', '', regex=True).astype(float)
df_Amazon['Open'] = df_Amazon['Open'].replace('[\$,]', '', regex=True).astype(float)
df_Amazon['High'] = df_Amazon['High'].replace('[\$,]', '', regex=True).astype(float)
df_Amazon['Low'] = df_Amazon['Low'].replace('[\$,]', '', regex=True).astype(float)
df_Amazon['Date']=pd.to_datetime(df_Amazon['Date'],errors='coerce')
df_Amazon = df_Amazon.sort_values(by="Date")
data_Amazon=pd.DataFrame(data=df_Amazon['Close/Last'],columns=['Close/Last'])
from sklearn.model_selection import train_test_split
data_Amazon_train,data_Amazon_test = train_test_split(data_Amazon, test_size=425/data_Amazon.shape[0],shuffle=False)
df_Amazon_train,df_Amazon_test = train_test_split(df_Amazon, test_size=365/df_Amazon.shape[0],shuffle=False)
scaler = MinMaxScaler()
data_Amazon_train=data_Amazon_train.values.reshape(-1,1)
data_Amazon_train= scaler.fit_transform(data_Amazon_train)
data_Amazon_test=data_Amazon_test.values.reshape(-1,1)
data_Amazon_test= scaler.transform(data_Amazon_test)
  
x_train = []
y_train = []
x_test = []
y_test = []
train_data_lstm= data_Amazon_train
test_data_lstm=data_Amazon_test
for i in range(60, len(train_data_lstm)):
  x_train.append(train_data_lstm[i-60:i, 0])
  y_train.append(train_data_lstm[i, 0])

x_train,y_train = np.array(x_train),np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
for i in range(60, len(test_data_lstm)):
  x_test.append(test_data_lstm[i-60:i, 0])
  y_test.append(test_data_lstm[i, 0])
x_test,y_test = np.array(x_test),np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
# Build the LSTM model
model = Sequential()
model.add(LSTM(120, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(60, return_sequences=False))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# Fit the Model
model.fit(x_train,y_train,epochs=40,batch_size=64,verbose=1)
y_predict = model.predict(x_test)

# Transform Back to the Original Data
y_predict = scaler.inverse_transform(y_predict)
y_predict=[j for i in y_predict for j in i]
y_test_Transform = scaler.inverse_transform(y_test.reshape(-1,1))

from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test_Transform,y_predict))
ma =mean_absolute_error(y_test_Transform,y_predict)
print("Root mean squared difference between actual and predicted currency of 365 days for Amazon is",rms)
print("Mean Absolute difference between actual and predicted currency of 365 days for Amazon is",ma)
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_Amazon.Date, y=df_Amazon['Close/Last'], name="Actual Value",
                         line_color='deepskyblue'))
fig.add_trace(go.Scatter(x=df_Amazon_test.Date, y=y_predict, name="Predicted Value",
                         line_color='dimgray'))
fig.update_layout(title_text='Amazon LSTM Forecasting',
                  xaxis_rangeslider_visible=True)
fig.show()

Epoch 1/40
32/32 [==============================] - 8s 142ms/step - loss: 0.0106
Epoch 2/40
32/32 [==============================] - 5s 145ms/step - loss: 3.9786e-04
Epoch 3/40
32/32 [==============================] - 4s 140ms/step - loss: 2.9042e-04
Epoch 4/40
32/32 [==============================] - 5s 142ms/step - loss: 2.9440e-04
Epoch 5/40
32/32 [==============================] - 5s 140ms/step - loss: 3.0798e-04
Epoch 6/40
32/32 [==============================] - 5s 141ms/step - loss: 2.7409e-04
Epoch 7/40
32/32 [==============================] - 5s 142ms/step - loss: 2.7952e-04
Epoch 8/40
32/32 [==============================] - 5s 142ms/step - loss: 2.4945e-04
Epoch 9/40
32/32 [==============================] - 5s 142ms/step - loss: 2.8340e-04
Epoch 10/40
32/32 [==============================] - 5s 141ms/step - loss: 2.4378e-04
Epoch 11/40
32/32 [==============================] - 4s 140ms/step - loss: 2.5252e-04
Epoch 12/40
32/32 [==============================] - 5s 143ms/step 

**Apple Prediction**

In [16]:
df_Apple['Close/Last'] = df_Apple['Close/Last'].replace('[\$,]', '', regex=True).astype(float)
df_Apple['Open'] = df_Apple['Open'].replace('[\$,]', '', regex=True).astype(float)
df_Apple['High'] = df_Apple['High'].replace('[\$,]', '', regex=True).astype(float)
df_Apple['Low'] = df_Apple['Low'].replace('[\$,]', '', regex=True).astype(float)
df_Apple['Date']=pd.to_datetime(df_Apple['Date'],errors='coerce')
df_Apple = df_Apple.sort_values(by="Date")
data_Apple=pd.DataFrame(data=df_Apple['Close/Last'],columns=['Close/Last'])
from sklearn.model_selection import train_test_split
data_Apple_train,data_Apple_test = train_test_split(data_Apple, test_size=425/data_Apple.shape[0],shuffle=False)
df_Apple_train,df_Apple_test = train_test_split(df_Apple, test_size=365/df_Apple.shape[0],shuffle=False)
scaler = MinMaxScaler()
data_Apple_train=data_Apple_train.values.reshape(-1,1)
data_Apple_train= scaler.fit_transform(data_Apple_train)
data_Apple_test=data_Apple_test.values.reshape(-1,1)
data_Apple_test= scaler.transform(data_Apple_test)
  
x_train = []
y_train = []
x_test = []
y_test = []
train_data_lstm= data_Apple_train
test_data_lstm=data_Apple_test
for i in range(60, len(train_data_lstm)):
  x_train.append(train_data_lstm[i-60:i, 0])
  y_train.append(train_data_lstm[i, 0])

x_train,y_train = np.array(x_train),np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
for i in range(60, len(test_data_lstm)):
  x_test.append(test_data_lstm[i-60:i, 0])
  y_test.append(test_data_lstm[i, 0])
x_test,y_test = np.array(x_test),np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
# Build the LSTM model
model = Sequential()
model.add(LSTM(120, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(60, return_sequences=False))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# Fit the Model
model.fit(x_train,y_train,epochs=40,batch_size=64,verbose=1)
y_predict = model.predict(x_test)

# Transform Back to the Original Data
y_predict = scaler.inverse_transform(y_predict)
y_predict=[j for i in y_predict for j in i]
y_test_Transform = scaler.inverse_transform(y_test.reshape(-1,1))

from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test_Transform,y_predict))
ma =mean_absolute_error(y_test_Transform,y_predict)
print("Root mean squared difference between actual and predicted currency of 365 days for Apple is",rms)
print("Mean Absolute difference between actual and predicted currency of 365 days for Apple is",ma)
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_Apple.Date, y=df_Apple['Close/Last'], name="Actual Value",
                         line_color='deepskyblue'))
fig.add_trace(go.Scatter(x=df_Apple_test.Date, y=y_predict, name="Predicted Value",
                         line_color='dimgray'))
fig.update_layout(title_text='Apple LSTM Forecasting',
                  xaxis_rangeslider_visible=True)
fig.show()

Epoch 1/40
32/32 [==============================] - 10s 134ms/step - loss: 0.0064
Epoch 2/40
32/32 [==============================] - 4s 134ms/step - loss: 3.6194e-04
Epoch 3/40
32/32 [==============================] - 4s 132ms/step - loss: 2.8086e-04
Epoch 4/40
32/32 [==============================] - 4s 134ms/step - loss: 2.7591e-04
Epoch 5/40
32/32 [==============================] - 4s 132ms/step - loss: 2.6113e-04
Epoch 6/40
32/32 [==============================] - 4s 132ms/step - loss: 2.6332e-04
Epoch 7/40
32/32 [==============================] - 4s 133ms/step - loss: 2.5648e-04
Epoch 8/40
32/32 [==============================] - 4s 132ms/step - loss: 2.7810e-04
Epoch 9/40
32/32 [==============================] - 4s 132ms/step - loss: 2.3044e-04
Epoch 10/40
32/32 [==============================] - 4s 131ms/step - loss: 2.4164e-04
Epoch 11/40
32/32 [==============================] - 4s 133ms/step - loss: 2.2760e-04
Epoch 12/40
32/32 [==============================] - 4s 133ms/step

**Meta Prediction**

In [13]:
df_Meta['Close/Last'] = df_Meta['Close/Last'].replace('[\$,]', '', regex=True).astype(float)
df_Meta['Open'] = df_Meta['Open'].replace('[\$,]', '', regex=True).astype(float)
df_Meta['High'] = df_Meta['High'].replace('[\$,]', '', regex=True).astype(float)
df_Meta['Low'] = df_Meta['Low'].replace('[\$,]', '', regex=True).astype(float)
df_Meta['Date']=pd.to_datetime(df_Meta['Date'],errors='coerce')
df_Meta = df_Meta.sort_values(by="Date")
data_Meta=pd.DataFrame(data=df_Meta['Close/Last'],columns=['Close/Last'])
from sklearn.model_selection import train_test_split
data_Meta_train,data_Meta_test = train_test_split(data_Meta, test_size=425/data_Meta.shape[0],shuffle=False)
df_Meta_train,df_Meta_test = train_test_split(df_Meta, test_size=365/df_Meta.shape[0],shuffle=False)
scaler = MinMaxScaler()
data_Meta_train=data_Meta_train.values.reshape(-1,1)
data_Meta_train= scaler.fit_transform(data_Meta_train)
data_Meta_test=data_Meta_test.values.reshape(-1,1)
data_Meta_test= scaler.transform(data_Meta_test)
  
x_train = []
y_train = []
x_test = []
y_test = []
train_data_lstm= data_Meta_train
test_data_lstm=data_Meta_test
for i in range(60, len(train_data_lstm)):
  x_train.append(train_data_lstm[i-60:i, 0])
  y_train.append(train_data_lstm[i, 0])

x_train,y_train = np.array(x_train),np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
for i in range(60, len(test_data_lstm)):
  x_test.append(test_data_lstm[i-60:i, 0])
  y_test.append(test_data_lstm[i, 0])
x_test,y_test = np.array(x_test),np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
# Build the LSTM model
model = Sequential()
model.add(LSTM(120, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(60, return_sequences=False))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# Fit the Model
model.fit(x_train,y_train,epochs=40,batch_size=64,verbose=1)
y_predict = model.predict(x_test)

# Transform Back to the Original Data
y_predict = scaler.inverse_transform(y_predict)
y_predict=[j for i in y_predict for j in i]
y_test_Transform = scaler.inverse_transform(y_test.reshape(-1,1))

from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test_Transform,y_predict))
ma =mean_absolute_error(y_test_Transform,y_predict)
print("Root mean squared difference between actual and predicted currency of 365 days for Meta is",rms)
print("Mean Absolute difference between actual and predicted currency of 365 days for Meta is",ma)
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_Meta.Date, y=df_Meta['Close/Last'], name="Actual Value",
                         line_color='deepskyblue'))
fig.add_trace(go.Scatter(x=df_Meta_test.Date, y=y_predict, name="Predicted Value",
                         line_color='dimgray'))
fig.update_layout(title_text='Meta LSTM Forecasting',
                  xaxis_rangeslider_visible=True)
fig.show()

Epoch 1/40
32/32 [==============================] - 8s 133ms/step - loss: 0.0199
Epoch 2/40
32/32 [==============================] - 4s 134ms/step - loss: 9.4204e-04
Epoch 3/40
32/32 [==============================] - 4s 136ms/step - loss: 5.6761e-04
Epoch 4/40
32/32 [==============================] - 4s 136ms/step - loss: 5.4883e-04
Epoch 5/40
32/32 [==============================] - 4s 135ms/step - loss: 5.5929e-04
Epoch 6/40
32/32 [==============================] - 4s 133ms/step - loss: 5.5832e-04
Epoch 7/40
32/32 [==============================] - 4s 135ms/step - loss: 5.2296e-04
Epoch 8/40
32/32 [==============================] - 4s 135ms/step - loss: 5.3744e-04
Epoch 9/40
32/32 [==============================] - 4s 134ms/step - loss: 5.2441e-04
Epoch 10/40
32/32 [==============================] - 4s 136ms/step - loss: 4.8367e-04
Epoch 11/40
32/32 [==============================] - 4s 135ms/step - loss: 4.7848e-04
Epoch 12/40
32/32 [==============================] - 4s 134ms/step 

**Google Prediction**

In [14]:
df_Google['Close/Last'] = df_Google['Close/Last'].replace('[\$,]', '', regex=True).astype(float)
df_Google['Open'] = df_Google['Open'].replace('[\$,]', '', regex=True).astype(float)
df_Google['High'] = df_Google['High'].replace('[\$,]', '', regex=True).astype(float)
df_Google['Low'] = df_Google['Low'].replace('[\$,]', '', regex=True).astype(float)
df_Google['Date']=pd.to_datetime(df_Google['Date'],errors='coerce')
df_Google = df_Google.sort_values(by="Date")
data_Google=pd.DataFrame(data=df_Google['Close/Last'],columns=['Close/Last'])
from sklearn.model_selection import train_test_split
data_Google_train,data_Google_test = train_test_split(data_Google, test_size=425/data_Google.shape[0],shuffle=False)
df_Google_train,df_Google_test = train_test_split(df_Google, test_size=365/df_Google.shape[0],shuffle=False)
scaler = MinMaxScaler()
data_Google_train=data_Google_train.values.reshape(-1,1)
data_Google_train= scaler.fit_transform(data_Google_train)
data_Google_test=data_Google_test.values.reshape(-1,1)
data_Google_test= scaler.transform(data_Google_test)
  
x_train = []
y_train = []
x_test = []
y_test = []
train_data_lstm= data_Google_train
test_data_lstm=data_Google_test
for i in range(60, len(train_data_lstm)):
  x_train.append(train_data_lstm[i-60:i, 0])
  y_train.append(train_data_lstm[i, 0])

x_train,y_train = np.array(x_train),np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
for i in range(60, len(test_data_lstm)):
  x_test.append(test_data_lstm[i-60:i, 0])
  y_test.append(test_data_lstm[i, 0])
x_test,y_test = np.array(x_test),np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
# Build the LSTM model
model = Sequential()
model.add(LSTM(120, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(60, return_sequences=False))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# Fit the Model
model.fit(x_train,y_train,epochs=40,batch_size=64,verbose=1)
y_predict = model.predict(x_test)

# Transform Back to the Original Data
y_predict = scaler.inverse_transform(y_predict)
y_predict=[j for i in y_predict for j in i]
y_test_Transform = scaler.inverse_transform(y_test.reshape(-1,1))

from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test_Transform,y_predict))
ma =mean_absolute_error(y_test_Transform,y_predict)
print("Root mean squared difference between actual and predicted currency of 365 days for Google is",rms)
print("Mean Absolute difference between actual and predicted currency of 365 days for Google is",ma)
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_Google.Date, y=df_Google['Close/Last'], name="Actual Value",
                         line_color='deepskyblue'))
fig.add_trace(go.Scatter(x=df_Google_test.Date, y=y_predict, name="Predicted Value",
                         line_color='dimgray'))
fig.update_layout(title_text='Google LSTM Forecasting',
                  xaxis_rangeslider_visible=True)
fig.show()

Epoch 1/40
25/25 [==============================] - 7s 137ms/step - loss: 0.0236
Epoch 2/40
25/25 [==============================] - 3s 134ms/step - loss: 0.0015
Epoch 3/40
25/25 [==============================] - 3s 135ms/step - loss: 9.6462e-04
Epoch 4/40
25/25 [==============================] - 3s 134ms/step - loss: 9.4470e-04
Epoch 5/40
25/25 [==============================] - 3s 133ms/step - loss: 9.0079e-04
Epoch 6/40
25/25 [==============================] - 3s 133ms/step - loss: 8.8683e-04
Epoch 7/40
25/25 [==============================] - 3s 135ms/step - loss: 8.6482e-04
Epoch 8/40
25/25 [==============================] - 3s 135ms/step - loss: 8.4180e-04
Epoch 9/40
25/25 [==============================] - 3s 133ms/step - loss: 8.4453e-04
Epoch 10/40
25/25 [==============================] - 3s 136ms/step - loss: 8.3120e-04
Epoch 11/40
25/25 [==============================] - 3s 135ms/step - loss: 7.7517e-04
Epoch 12/40
25/25 [==============================] - 3s 133ms/step - lo

**Netflix Prediction**

In [15]:
df_Netflix['Close/Last'] = df_Netflix['Close/Last'].replace('[\$,]', '', regex=True).astype(float)
df_Netflix['Open'] = df_Netflix['Open'].replace('[\$,]', '', regex=True).astype(float)
df_Netflix['High'] = df_Netflix['High'].replace('[\$,]', '', regex=True).astype(float)
df_Netflix['Low'] = df_Netflix['Low'].replace('[\$,]', '', regex=True).astype(float)
df_Netflix['Date']=pd.to_datetime(df_Netflix['Date'],errors='coerce')
df_Netflix = df_Netflix.sort_values(by="Date")
data_Netflix=pd.DataFrame(data=df_Netflix['Close/Last'],columns=['Close/Last'])
from sklearn.model_selection import train_test_split
data_Netflix_train,data_Netflix_test = train_test_split(data_Netflix, test_size=425/data_Netflix.shape[0],shuffle=False)
df_Netflix_train,df_Netflix_test = train_test_split(df_Netflix, test_size=365/df_Netflix.shape[0],shuffle=False)
scaler = MinMaxScaler()
data_Netflix_train=data_Netflix_train.values.reshape(-1,1)
data_Netflix_train= scaler.fit_transform(data_Netflix_train)
data_Netflix_test=data_Netflix_test.values.reshape(-1,1)
data_Netflix_test= scaler.transform(data_Netflix_test)
  
x_train = []
y_train = []
x_test = []
y_test = []
train_data_lstm= data_Netflix_train
test_data_lstm=data_Netflix_test
for i in range(60, len(train_data_lstm)):
  x_train.append(train_data_lstm[i-60:i, 0])
  y_train.append(train_data_lstm[i, 0])

x_train,y_train = np.array(x_train),np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
for i in range(60, len(test_data_lstm)):
  x_test.append(test_data_lstm[i-60:i, 0])
  y_test.append(test_data_lstm[i, 0])
x_test,y_test = np.array(x_test),np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
# Build the LSTM model
model = Sequential()
model.add(LSTM(120, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(60, return_sequences=False))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# Fit the Model
model.fit(x_train,y_train,epochs=40,batch_size=64,verbose=1)
y_predict = model.predict(x_test)

# Transform Back to the Original Data
y_predict = scaler.inverse_transform(y_predict)
y_predict=[j for i in y_predict for j in i]
y_test_Transform = scaler.inverse_transform(y_test.reshape(-1,1))

from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test_Transform,y_predict))
ma =mean_absolute_error(y_test_Transform,y_predict)
print("Root mean squared difference between actual and predicted currency of 365 days for Netflix is",rms)
print("Mean Absolute difference between actual and predicted currency of 365 days for Netflix is",ma)
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_Netflix.Date, y=df_Netflix['Close/Last'], name="Actual Value",
                         line_color='deepskyblue'))
fig.add_trace(go.Scatter(x=df_Netflix_test.Date, y=y_predict, name="Predicted Value",
                         line_color='dimgray'))
fig.update_layout(title_text='Netflix LSTM Forecasting',
                  xaxis_rangeslider_visible=True)
fig.show()

Epoch 1/40
32/32 [==============================] - 9s 134ms/step - loss: 0.0198
Epoch 2/40
32/32 [==============================] - 4s 134ms/step - loss: 7.7154e-04
Epoch 3/40
32/32 [==============================] - 4s 135ms/step - loss: 4.9463e-04
Epoch 4/40
32/32 [==============================] - 4s 134ms/step - loss: 4.7779e-04
Epoch 5/40
32/32 [==============================] - 4s 135ms/step - loss: 5.1052e-04
Epoch 6/40
32/32 [==============================] - 4s 134ms/step - loss: 4.6235e-04
Epoch 7/40
32/32 [==============================] - 4s 136ms/step - loss: 4.6361e-04
Epoch 8/40
32/32 [==============================] - 4s 135ms/step - loss: 4.6138e-04
Epoch 9/40
32/32 [==============================] - 4s 136ms/step - loss: 4.6387e-04
Epoch 10/40
32/32 [==============================] - 4s 133ms/step - loss: 4.1537e-04
Epoch 11/40
32/32 [==============================] - 4s 134ms/step - loss: 4.1254e-04
Epoch 12/40
32/32 [==============================] - 4s 134ms/step 